### Compute base depth values for NuScenes (used in PGD config)

In [9]:
import json
import os
import numpy as np 
import copy
import tempfile
import warnings
from os import path as osp

import mmcv
import numpy as np
import pyquaternion
import torch
from nuscenes.utils.data_classes import Box as NuScenesBox
import mmcv
# from mmdet3d.core import bbox3d2result, box3d_multiclass_nms, xywhr2xyxyr
# from mmdet.datasets import CocoDataset

In [46]:
val_anno_file = '/home/anishmad/msr_thesis/mmdet3d-lt3d/data/nuscenes/nuscenes_infos_val_mono3d.coco.json'
train_anno_file = '/home/anishmad/msr_thesis/mmdet3d-lt3d/data/nuscenes/nuscenes_infos_train_mono3d.coco.json'

In [28]:
class_names = [
'car', 'truck', 'trailer', 'bus', 'construction_vehicle', 'bicycle', 'motorcycle', 'emergency_vehicle',
'adult', 'child', 'police_officer', 'construction_worker', 'stroller', 'personal_mobility', 
'pushable_pullable', 'debris', 'traffic_cone', 'barrier'
]

In [51]:
data_train = json.load(open(train_anno_file,'r'))
data_val = json.load(open(val_anno_file,'r'))

In [55]:
classwise_depths = defaultdict(list)

def get_mean_depths(data):
    mean_std_depths = []
    for idx,anno in enumerate(data['annotations']):
        classwise_depths[anno['category_name']].append(anno['center2d'][2])
        # l.append(anno['category_name'])

    for cat, cat_depths in classwise_depths.items():
        # print(cat, np.mean(cat_depths), np.std(cat_depths))
        mean, std = np.round(np.mean(cat_depths),3), np.round(np.std(cat_depths),3)
        mean_std_depths.append((mean,std))
    print(mean_std_depths)
    return mean_std_depths

print('Train\n')
ms_depth_train = get_mean_depths(data_train)
print('Val\n')
ms_depth_val = get_mean_depths(data_val)

Train

[(37.152, 24.632), (31.99, 21.124), (20.606, 13.679), (23.893, 15.209), (20.571, 14.341), (34.157, 20.107), (27.457, 15.528), (22.736, 15.011), (22.193, 16.328), (24.278, 16.049), (22.348, 13.704), (40.911, 26.341), (39.687, 23.974), (22.298, 10.944), (24.985, 12.478), (29.132, 16.155), (18.995, 12.011), (29.624, 21.013)]
Val

[(37.452, 24.777), (32.182, 21.19), (20.319, 13.441), (23.526, 14.964), (20.533, 14.41), (35.169, 20.511), (27.441, 15.706), (23.116, 15.582), (23.466, 16.964), (24.47, 15.695), (22.391, 13.626), (40.499, 26.208), (40.022, 23.644), (22.113, 10.887), (25.227, 12.822), (27.922, 16.527), (19.102, 11.823), (30.01, 21.656)]


### Compute Base Dims for NuScenes dataset (used in PGD config)

In [58]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version="v1.0-trainval", dataroot='/ssd0/nperi/nuScenes/', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 45.786 seconds.
Reverse indexing ...
Done reverse indexing in 10.5 seconds.


In [59]:
nusc.list_categories()

Category stats for split v1.0-trainval:
animal                      n=  787, width= 0.37±0.13, len= 0.86±0.36, height= 0.60±0.20, lw_aspect= 2.35±0.69
human.pedestrian.adult      n=208240, width= 0.67±0.13, len= 0.73±0.19, height= 1.77±0.18, lw_aspect= 1.11±0.26
human.pedestrian.child      n= 2066, width= 0.51±0.14, len= 0.53±0.15, height= 1.38±0.25, lw_aspect= 1.05±0.23
human.pedestrian.constructi n= 9161, width= 0.72±0.20, len= 0.71±0.20, height= 1.74±0.30, lw_aspect= 1.02±0.29
human.pedestrian.personal_m n=  395, width= 0.62±0.12, len= 1.18±0.31, height= 1.71±0.27, lw_aspect= 1.98±0.64
human.pedestrian.police_off n=  727, width= 0.73±0.14, len= 0.69±0.13, height= 1.83±0.14, lw_aspect= 0.97±0.18
human.pedestrian.stroller   n= 1072, width= 0.63±0.13, len= 0.95±0.27, height= 1.17±0.15, lw_aspect= 1.58±0.68
human.pedestrian.wheelchair n=  503, width= 0.77±0.10, len= 1.09±0.23, height= 1.37±0.09, lw_aspect= 1.42±0.23
movable_object.barrier      n=152087, width= 2.53±0.64, len= 0.50±0.17,

#### Manually copied over values

In [62]:
#### All classes listed as given in config https://github.com/neeharperi/mmdet3d-lt3d/blob/main/configs/_base_/datasets/nus-mono3d.py 
# class_names = [
# 'car', 'truck', 'trailer', 'bus', 'construction_vehicle', 'bicycle', 'motorcycle', 'emergency_vehicle',
# 'adult', 'child', 'police_officer', 'construction_worker', 'stroller', 'personal_mobility', 
# 'pushable_pullable', 'debris', 'traffic_cone', 'barrier'
# ]

## follow (len, height, width) convention.  There are slight differences in values probably due to using trainval set and not simply train set.
## If slighlty different values are an issue, use method for base_depths to compute base_dims over the train set.

base_dims = { 'car': (4.62, 1.73, 1.96), 'truck': (6.93, 2.83, 2.51), 'trailer': (12.56, 3.89, 2.94), 'bus': (11.22, 3.50, 2.95),
            'construction_vehicle': (6.68, 3.21, 2.85), 'bicycle': (1.70, 1.28, 0.60), 'motorcycle': (2.11, 1.46, 0.78),
             'emergency_vehicle': (5.04, 1.85, 2.03), 'adult': (0.73, 1.77, 0.67), 'child': (0.53, 1.38, 0.51), 
             'police_officer': (0.69, 1.83, 0.73), 'construction_worker': (0.71, 1.74, 0.72), 'stroller': (0.95, 1.17, 0.63), 
             'personal_mobility': (1.18, 1.71, 0.62), 'pushable_pullable': (0.67, 1.06, 0.60), 'debris': (1.08, 1.26, 1.01), 
             'traffic_cone': (0.41, 1.07, 0.41), 'barrier':(0.5, 0.98, 2.53)
            }

# classes not included
## animal,  wheelchair, bicycle_rack, bendy_bus, (only taken emergency vehicle as police vehicle, not ambulance), 

In [61]:
list(base_dims.values())

[(4.62, 1.73, 1.96),
 (6.93, 2.83, 2.51),
 (12.56, 3.89, 2.94),
 (11.22, 3.5, 2.95),
 (6.68, 3.21, 2.85),
 (1.7, 1.28, 0.6),
 (2.11, 1.46, 0.78),
 (5.04, 1.85, 2.03),
 (0.73, 1.77, 0.67),
 (0.53, 1.38, 0.51),
 (0.69, 1.83, 0.73),
 (0.71, 1.74, 0.72),
 (0.95, 1.17, 0.63),
 (1.18, 1.71, 0.62),
 (0.67, 1.06, 0.6),
 (1.08, 1.26, 1.01),
 (0.41, 1.07, 0.41),
 (0.5, 0.98, 2.53)]